In [533]:
import numpy as np
import pandas as pd

MAXEPOCH = 1000

In [534]:
file = open("multiclass_train.txt")

lines = file.readlines()

numClass, numFeature, datasetLen = 0, 0, 0

dataset = []

count = 0
for line in lines:
    if count == 0:
        var = line.split()
        numClass   = int(var[0])
        numFeature = int(var[1])
        datasetLen = int(var[2])
    else:
        var = line.split()
        data = []
        for i in range(numFeature):
            data.append(float(var[i]))
        data.append(int(var[numFeature]))
        dataset.append(data)
 
    count += 1

In [535]:
file = open("multiclass_test.txt")

lines = file.readlines()
test_dataset = []

for line in lines:
    var = line.split()
    data = []
    for i in range(numFeature):
        data.append(float(var[i]))
    data.append(int(var[numFeature]))
    test_dataset.append(data)

In [536]:
def test(dataset,w):
    count_accurate = 0
    for data in dataset:
        x = np.array(data)
        group = x[numFeature]
        x[numFeature] = 1
        x = np.array(x)
        x = x.reshape(numFeature+1,1)
        dot_product = np.dot(w,x)[0]
        predicted = -1
        if dot_product >= 0:
            predicted = 1
        else:
            predicted = 2

        if predicted==group:
            count_accurate += 1

    print("Accuracy :",float((count_accurate/len(dataset))*100))
    
    return float(count_accurate/len(dataset))

In [537]:
weights = []

for i in range(numClass):
    wi = np.random.uniform(-10,10,numFeature+1)
    weights.append(wi)

In [538]:
print(weights)

[array([-8.71252052,  1.66273518,  0.13559253,  6.20948245]), array([ 5.06717674, -9.25807956, -6.94842691, -4.32583755]), array([-6.11542058, -6.78322061, -4.31463162,  2.44270877])]


In [539]:
# classwise dataset extraction

df = pd.DataFrame(dataset)

In [540]:
classes = np.unique(df[numFeature])

In [541]:
classes

array([1, 2, 3])

In [542]:
class_wise_dataset = []

for i in classes:
    each_dataset = df.where(
                df[numFeature] == i).dropna()
    class_wise_dataset.append(each_dataset)

In [543]:
w = []
for each in weights:
    for e in each:
        w.append(e)

In [544]:
w = np.array(w)
w.shape

(12,)

In [545]:
dataset = []
for i in range(numClass):
    for j in range(numClass):
        if i != j:
            row = np.zeros(numClass)
            row[i] = 1
            row[j] = -1
            dataset.append(row)
print(dataset)
            

[array([ 1., -1.,  0.]), array([ 1.,  0., -1.]), array([-1.,  1.,  0.]), array([ 0.,  1., -1.]), array([-1.,  0.,  1.]), array([ 0., -1.,  1.])]


In [546]:
def zero_array(numClass,size):
    arr = []
    for i in range(numClass):
        each = np.zeros(size)
        arr.append(each)
    return np.array(arr)

dataset = []
count = 0
for ds in class_wise_dataset:
    values = ds.values
    for each in values:
        each[numFeature] = 1
        for j in range(numClass):
            if count != j:
                row = zero_array(numClass,len(each))
                row[count] = each
                row[j] = -each
                row = row.reshape((numClass*(numFeature+1)),1)
                dataset.append(row)
    
    count +=1

In [547]:
learning_rate = 0.025

In [548]:
t = 0

In [549]:
for i in range(MAXEPOCH):
    count = 0 
    for i in range(len(dataset)):
        dot_product = np.dot(w,dataset[i])[0]
        if dot_product<=0:
            count += 1
            w = w + learning_rate*np.array(dataset[i].reshape((numClass*(numFeature+1)),))
        else:
            pass
    if count==0:
        break

In [550]:
w = w.reshape(numClass,(numFeature+1))

In [551]:
print(w)

[[ -5.56983552  -3.46259482  -0.60856997   4.18448245]
 [  3.59247924 -10.52621456  -7.75880941  -4.50083755]
 [ -7.78340808  -0.38975561  -2.76008662   4.64270877]]


In [552]:
count_accurate = 0

for i in test_dataset:
    group = i[numFeature]
    i[numFeature] = 1
    i = np.array(i)
    i = i.reshape(numFeature+1,1)
    dot_sum = []
    for wi in w:
        dot_product = np.dot(wi,i)[0]
        dot_sum.append(dot_product)
    pred = np.argmax(dot_sum) + 1
    
    if pred==group:
        count_accurate += 1
    
print("Accuracy :",float((count_accurate/len(test_dataset))*100))

Accuracy : 98.33333333333333
